In [5]:
data = pd.read_csv('Data/quant_trading_data.txt')

In [ ]:
class MeanStrategy1(strategy.BacktestingStrategy):

    # 2.1. Define Strategy Initialization Function
    def __init__(self, feed, instrument, nper, nstdev):
        super(MeanStrategy1, self).__init__(feed, 10000)
        self.position = None
        self.instrument = instrument
        self.setUseAdjustedValues(True)
        self.bbands = bollinger.BollingerBands(feed[instrument].getPriceDataSeries(), nper, nstdev)

    # 2.2. Define Get Technical Indicators Functions
    def getBBands(self):
        return self.bbands

    # 2.3. Define Overriding Strategy Functions

    # onEnterOk: Get notified when order submitted to enter a position was filled
    def onEnterOk(self, position):
        tradeInfo = position.getEntryOrder().getExecutionInfo()
        self.info('Buy %s shares at $%.2f' % (self.shares, tradeInfo.getPrice()))
        # Stop-Loss Exit Order
        # (Instructions: Un-comment Stop-Loss Exit Order)
        # self.position.exitStop(tradeInfo.getPrice() * 0.95, goodTillCanceled=True)

    # onEnterCanceled: Get notified when order submitted to enter a position was canceled and update position
    def onEnterCanceled(self, position):
        self.position = None

    # onExitOk: Get notified when order submitted to exit a position was filled and update position
    def onExitOk(self, position):
        tradeInfo = position.getExitOrder().getExecutionInfo()
        self.info('Sell %s shares at $%.2f' % (self.shares, tradeInfo.getPrice()))
        # Stop-Loss Exit Order Execution Information
        # (Instructions: uncomment stop-loss exit order information, comment market exit order information)
        # if position.getExitOrder().getType() == broker.Order.Type.STOP:
        #     orderType = 'stop'
        # else:
        #     orderType = 'market'
        # self.info('Sell %s shares at $%.2f with a %s order' % (self.shares, tradeInfo.getPrice(), orderType))
        self.position = None

    # onExitCanceled: Get notified when order submitted to exit a position was canceled.
    # Re-submit order when canceled
    def onExitCanceled(self, position):
         self.position.exitMarket()

    # 2.4. Define Trading Strategy Function

    # Mean-Reversion Strategy
    # Enter Long Order = Buy when Close < Lower Band
    # Exit Order = Sell when Close > Upper Band

    def onBars(self, bars):
        if self.bbands.getLowerBand()[-1] is None:
            return

        if self.position is None:
            if bars[self.instrument].getPrice() < self.bbands.getLowerBand()[-1]:
                # 95% equity investment for difference between order day Close price and next day Open price
                # number of shares can also be a fixed quantity for all transactions (ex. self.shares = 10)
                self.shares = int(self.getBroker().getCash() * 0.95 / bars[self.instrument].getPrice())
                self.position = self.enterLong(self.instrument, self.shares, goodTillCanceled=True)

        elif bars[self.instrument].getPrice() > self.bbands.getUpperBand()[-1] and not self.position.exitActive():
            self.position.exitMarket()
        # Stop-Loss Exit Order Cancelling
        # (Instructions: uncomment stop-loss exit order cancelling, comment market exit order)
        # elif bars[self.instrument].getPrice() > self.bbands.getUpperBand()[-1]:
        #     self.position.cancelExit()

# 3. Define Run Strategy Function

def MeanStrategyRun1(nper, nstdev):

    # 3.1. Create Instruments object with stock tickers
    instruments = ['SPY']

    # 3.2. Load CSV Feed previously downloaded or read
    feed = csvfeed.GenericBarFeed(bar.Frequency.DAY)
    feed.addBarsFromCSV(instruments[0], 'Data//Quantitative-Trading-Analysis-Data.txt', skipMalformedBars=True)
    # Optimized Strategy Testing
    # (Instructions: uncomment testing data range CSV feed loading, comment full data range CSV feed loading)
    # feed.addBarsFromCSV(instruments[0], 'Data//Quantitative-Trading-Analysis-Data-Testing.txt',
    #                     skipMalformedBars=True)

    # 3.3. Evaluate Strategy with CSV Feed and Technical Indicator Parameters
    meanStrategy1 = MeanStrategy1(feed, instruments[0], nper, nstdev)

    # 3.4. Set Strategy Commission
    # FixedPerTrade() or TradePercentage()
    meanStrategy1.getBroker().setCommission(broker.backtesting.FixedPerTrade(6))

    # 3.5. Attach Strategy Trading Statistics Analyzers
    retAnalyzer = ret.Returns(maxLen=2518)
    meanStrategy1.attachAnalyzer(retAnalyzer)
    sharpeRatioAnalyzer = sharpe.SharpeRatio()
    meanStrategy1.attachAnalyzer(sharpeRatioAnalyzer)
    drawDownAnalyzer = drawdown.DrawDown()
    meanStrategy1.attachAnalyzer(drawDownAnalyzer)
    tradesAnalyzer = trades.Trades()
    meanStrategy1.attachAnalyzer(tradesAnalyzer)

    # 3.6. Attach Strategy Plotter

    # Buy and Sell Chart
    plt1 = plotter.StrategyPlotter(meanStrategy1, plotPortfolio=False)
    plt1.getInstrumentSubplot('SPY').addDataSeries('Lower', meanStrategy1.getBBands().getLowerBand())
    plt1.getInstrumentSubplot('SPY').addDataSeries('Middle', meanStrategy1.getBBands().getMiddleBand())
    plt1.getInstrumentSubplot('SPY').addDataSeries('Upper', meanStrategy1.getBBands().getUpperBand())

    # Equity Curve Chart
    plt2 = plotter.StrategyPlotter(meanStrategy1, plotAllInstruments=False)

    # Cumulative Returns Chart
    plt3 = plotter.StrategyPlotter(meanStrategy1, plotAllInstruments=False, plotPortfolio=False)
    plt3.getOrCreateSubplot('Cumulative returns').addDataSeries('Mean1', retAnalyzer.getCumulativeReturns())
    plt3.getOrCreateSubplot('Cumulative returns').addDataSeries('B&H', cumret.CumulativeReturn(
        feed['SPY'].getPriceDataSeries()))

    # 3.7. Run Strategy
    start = time.time()
    meanStrategy1.run()
    end = time.time()

    # 3.8. Print Strategy Trading Statistics
    print('')
    print('------------------------------')
    print('')
    print('1. Portfolio statistics')
    print('')
    print('1.1. Portfolio p&l')
    print('')
    print('Portfolio initial equity: $10000.00')
    print('Portfolio final equity: $%.2f' % meanStrategy1.getBroker().getEquity())
    print('Portfolio net trading p&l: $%.2f' % tradesAnalyzer.getAll().sum())
    print('Portfolio maximum drawdown: %.2f %%' % (drawDownAnalyzer.getMaxDrawDown() * 100))
    print('')
    print('1.2. Portfolio performance')
    print('')
    print('Portfolio cumulative return: %.2f %%' % (retAnalyzer.getCumulativeReturns()[-1] * 100))
    print('Portfolio annualized return: %.2f %%' % (((retAnalyzer.getCumulativeReturns()[-1] + 1) **
                                                     (252.0 / 2518.0) - 1) * 100))
    print('Portfolio annualized mean return: %.2f %%' % (stats.mean(retAnalyzer.getReturns()) * 252 * 100))
    print('Portfolio annualized standard deviation: %.2f %%' % (stats.stddev(retAnalyzer.getReturns()) * np.sqrt(252)
                                                                * 100))
    print('Portfolio annualized Sharpe ratio (Rf = 0%%): %.2f' % (sharpeRatioAnalyzer.getSharpeRatio(riskFreeRate=0.0,
                                                                                                     annualized=True)))
    print('')
    print('------------------------------')

    print('')
    print('2. Total trades statistics')
    print('')
    print('Total trades: %d' % (tradesAnalyzer.getCount()))
    if tradesAnalyzer.getCount() > 0:
        tradesProfits = tradesAnalyzer.getAll()
        print('')
        print('2.1. Total trades p&l')
        print('')
        print('Total trades average p&l: $%2.f' % (tradesProfits.mean()))
        print('Total trades p&l standard deviation: $%2.f' % (tradesProfits.std()))
        print('Total trades maximum p&l: $%2.f' % (tradesProfits.max()))
        print('Total trades minimum p&l: $%2.f' % (tradesProfits.min()))
        tradesReturns = tradesAnalyzer.getAllReturns()
        print('')
        print('2.2. Total trades returns')
        print('')
        print('Total trades average return: %2.f %%' % (tradesReturns.mean() * 100))
        print('Total trades returns standard deviation: %2.f %%' % (tradesReturns.std() * 100))
        print('Total trades maximum return: %2.f %%' % (tradesReturns.max() * 100))
        print('Total trades minimum return: %2.f %%' % (tradesReturns.min() * 100))
    print('')
    print('------------------------------')

    print('')
    print("3. Profitable trades statistics")
    print('')
    print('Profitable trades: %d' % (tradesAnalyzer.getProfitableCount()))
    if tradesAnalyzer.getProfitableCount() > 0:
        profTradesProfits = tradesAnalyzer.getProfits()
        print('')
        print('3.1. Profitable trades p&l')
        print('')
        print('Profitable trades average profit: $%2.f' % (profTradesProfits.mean()))
        print('Profitable trades profits standard deviation: $%2.f' % (profTradesProfits.std()))
        print('Profitable trades maximum profit: $%2.f' % (profTradesProfits.max()))
        print('Profitable trades minimum profit: $%2.f' % (profTradesProfits.min()))
        profTradesReturns = tradesAnalyzer.getPositiveReturns()
        print('')
        print('3.2. Profitable trades returns')
        print('')
        print('Profitable trades average return: %2.f %%' % (profTradesReturns.mean() * 100))
        print('Profitable trades returns standard deviation: %2.f %%' % (profTradesReturns.std() * 100))
        print('Profitable trades maximum return: %2.f %%' % (profTradesReturns.max() * 100))
        print('Profitable trades minimum return: %2.f %%' % (profTradesReturns.min() * 100))
    print('')
    print('------------------------------')

    print('')
    print('4. Unprofitable trades statistics')
    print('')
    print('Unprofitable trades: %d' % (tradesAnalyzer.getUnprofitableCount()))
    if tradesAnalyzer.getUnprofitableCount() > 0:
        unprofTradesLosses = tradesAnalyzer.getLosses()
        print('')
        print('4.1. Unprofitable trades p&l')
        print('')
        print('Unprofitable trades average loss: $%2.f' % (unprofTradesLosses.mean()))
        print('Unprofitable trades losses standard deviation: $%2.f' % (unprofTradesLosses.std()))
        print('Unprofitable trades maximum loss: $%2.f' % (unprofTradesLosses.min()))
        print('Unprofitable trades minimum loss: $%2.f' % (unprofTradesLosses.max()))
        unprofTradesReturns = tradesAnalyzer.getNegativeReturns()
        print('')
        print('4.2. Unprofitable trades returns')
        print('')
        print('Unprofitable trades average return: %2.f %%' % (unprofTradesReturns.mean() * 100))
        print('Unprofitable trades returns standard deviation: %2.f %%' % (unprofTradesReturns.std() * 100))
        print('Unprofitable trades maximum return: %2.f %%' % (unprofTradesReturns.max() * 100))
        print('Unprofitable trades minimum return: %2.f %%' % (unprofTradesReturns.min() * 100))
    print('')
    print("------------------------------")
    print('')
    print('Strategy Running Time: ', round(end - start, 4), ' seconds')

    # 3.9. Plot Strategy
    plt1.plot(fromDateTime=dt.datetime(2016, 1, 1), toDateTime=dt.datetime(2016, 12, 31))
    plt2.plot()
    plt3.plot()

MeanStrategyRun1(20, 2)

# Optimized Strategy Testing
# (Instructions: uncomment optimized strategy testing run, comment strategy run)
# MeanStrategyRun1(15, 2)

In [12]:
import pyalgotrade.barfeed.csvfeed as csvfeed
import pyalgotrade.bar as bar
import pyalgotrade.optimizer.local as local
import pyalgotrade.strategy as strategy
import pyalgotrade.technical.bollinger as bollinger
import itertools
import time

# 2. Create Strategy Optimization Class

class MeanStrategy1(strategy.BacktestingStrategy):

    # 2.1. Define Strategy Initialization Function
    def __init__(self, feed, nper, nstdev):
        super(MeanStrategy1, self).__init__(feed, 10000)
        self.position = None
        self.instrument = 'SPY'
        self.setUseAdjustedValues(True)
        self.bbands = bollinger.BollingerBands(feed['SPY'].getPriceDataSeries(), nper, nstdev)

    # 2.2. Define Get Technical Indicators Functions
    def getBBands(self):
        return self.bbands

    # 2.3. Define Overriding Strategy Functions

    # onEnterOk: Get notified when order submitted to enter a position was filled
    def onEnterOk(self, position):

        # Market Enter Order Trade Execution Information
        tradeInfo = position.getEntryOrder().getExecutionInfo()
        self.info('Buy %s shares at $%.2f' % (self.shares, tradeInfo.getPrice()))
        # Stop-Loss Exit Order
        # (Instructions: Un-comment Stop-Loss Exit Order)
        # self.position.exitStop(tradeInfo.getPrice() * 0.95, True)

    # onEnterCanceled: Get notified when order submitted to enter a position was canceled and update position
    def onEnterCanceled(self, position):
        self.position = None

    # onExitOk: Get notified when order submitted to exit a position was filled and update position
    def onExitOk(self, position):
        tradeInfo = position.getExitOrder().getExecutionInfo()

        # Market Exit Order Execution Information
        self.info('Sell %s shares at $%.2f' % (self.shares, tradeInfo.getPrice()))
        # Stop-Loss Exit Order Execution Information
        # (Instructions: uncomment stop-loss exit order information, comment market exit order information)
        # if position.getExitOrder().getType() == broker.Order.Type.STOP:
        #     orderType = "stop"
        # else:
        #     orderType = "market"
        # self.info('Sell %s shares at $%.2f with a %s order' % (self.shares, tradeInfo.getPrice(), orderType))
        self.position = None

    # onExitCanceled: Get notified when order submitted to exit a position was canceled.
    # Re-submit order when canceled
    def onExitCanceled(self, position):
         self.position.exitMarket()

    # 2.4. Define Trading Strategy Function

    # Mean-Reversion Strategy
    # Enter Long Order = Buy when Close < Lower Band
    # Exit Order = Sell when Close > Upper Band

    def onBars(self, bars):
        if self.bbands.getLowerBand()[-1] is None:
            return

        if self.position is None:
            if bars[self.instrument].getPrice() < self.bbands.getLowerBand()[-1]:
                # 95% equity investment for difference between order day Close price and next day Open price
                # number of shares can also be a fixed quantity for all transactions (ex. self.shares = 10)
                self.shares = int(self.getBroker().getCash() * 0.95 / bars[self.instrument].getPrice())
                self.position = self.enterLong(self.instrument, self.shares, goodTillCanceled=True)

        elif bars[self.instrument].getPrice() > self.bbands.getUpperBand()[-1] and not self.position.exitActive():
            self.position.exitMarket()
        # Stop-Loss Exit Order Cancelling
        # (Instructions: uncomment stop-loss exit order cancelling, comment market exit order)
        # elif bars[self.instrument].getPrice() > self.bbands.getUpperBand()[-1]:
        #     self.position.cancelExit()

In [ ]:

# 3. Do Strategy Optimization

# 3.1. Define technical indicators parameters combinations
def parameters():
    nper = (15, 20, 25)
    nstdev = (1, 2, 3)
    return itertools.product(nper, nstdev)

# 3.2. Run strategy optimization
# if__name__ == '__main__': only needed for Windows
if __name__ == '__main__':
    instruments = ['SPY']

    # Load CSV Feed previously downloaded or read
    feed = csvfeed.GenericBarFeed(bar.Frequency.DAY)
    feed.addBarsFromCSV(instruments[0], "Data/quant_trading_data.txt", skipMalformedBars=True)

    # Strategy Optimization
    start = time.time()
    local.run(MeanStrategy1, feed, parameters())
    end = time.time()

    print("------------------------------")
    print('Strategy Optimization Time: ', round(end - start, 4), ' seconds')

2018-10-09 15:26:40,213 pyalgotrade.optimizer.local [INFO] Starting server on port 15195
2018-10-09 15:26:40,215 pyalgotrade.optimizer.xmlrpcserver [INFO] Loading bars
2018-10-09 15:26:40,227 pyalgotrade.optimizer.local [INFO] Waiting for the server to be ready
2018-10-09 15:26:40,384 pyalgotrade.optimizer.xmlrpcserver [INFO] Started serving
2018-10-09 15:26:40,386 pyalgotrade.optimizer.local [INFO] Starting 4 workers
